# Generation and Calibration (3compartment -> no_arm_CVS)

This notebook walks through the end-to-end workflow for generating, simulating, and calibrating a 3compartment model, then extending to a 0D-1D hybrid.

## Steps overview
1. Open vessel array file for 3compartment (switch to no_arm_CVS) in PhLynx.
2. Generate CellML model from PhLynx (saved to Downloads).
3. Set up `user_inputs_dict` for running the model.
4. Load the model with `SimulationHelper`.
5. Simulate the model and plot outputs.
6. Plot ground-truth data vs. uncalibrated outputs.
7. Add arm vessels in PhLynx and regenerate.
8. Create `obs_data.json` from Python utilities.
9. Create `params_for_id.csv` from Python.
10. Run sensitivity analysis over a large parameter set.
11. Choose the most influential parameters (e.g., top 2).
12. Create a `param_id` object.
13. Calibrate with the genetic algorithm.
14. View auto-generated calibration plots.
15. Create additional plots from saved outputs.
16. Calculate PWV with the 0D model.
17. Switch selected vessels to 1D.
18. Generate a new C++ model with 1D vessels.
19. Run the C++ model.
20. Load 0D-1D hybrid results.
21. Plot 0D-1D vs 0D results.
22. Compute PWV for the hybrid and compare to 0D.

> Update paths and filenames to match your environment.


## 0) Set up imports and paths

In [ ]:
# Core imports
from pathlib import Path
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
try:
    import opencor as oc
except:
    print('opencor not available, open this jupyter notebook with a python version that has opencor installed')
    exit()

# Ensure local src is importable
project_root = Path("/home/farg967/Documents/git_projects/circulatory_autogen")
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# Set up paths
resources_dir = project_root / "resources"
generated_models_dir = project_root / "generated_models"
param_id_output_dir = project_root / "param_id_output"
downloads_dir = Path.home() / "Downloads"

## 1) Open vessel array in PhLynx (manual)

- Open the 3compartment  vessel array in PhLynx [LINK_TO_PHLYNX].
- Save/export the vessel array so it is ready for CellML generation.

## 2) Generate CellML in PhLynx (manual)

- In PhLynx, generate the CellML model for the edited vessel array.
- The exported file is typically saved to `~/Downloads`.
- Note the exported filename (e.g., `3compartment.cellml`). 

In [ ]:

# Model identifiers
file_prefix = "3compartment"  
input_param_file = f"{file_prefix}_parameters.csv"

# Base user inputs (extend as needed later)
user_inputs_dict = {
    "DEBUG": True,
    "file_prefix": file_prefix,
    "input_param_file": input_param_file,
    "model_type": "cellml_only",  # options: cellml_only, python, cpp
    "solver": "CVODE",
    "solver_info": {
        "MaximumStep": 0.001,
        "MaximumNumberOfSteps": 5000,
    },
    "dt": 0.01,
    "resources_dir": str(resources_dir),
    "generated_models_dir": str(generated_models_dir),
    "param_id_output_dir": str(param_id_output_dir),
    "param_id_method": "genetic_algorithm",
    "couple_to_1d": False,
    "cpp_generated_models_dir": str(generated_models_dir / file_prefix),
}

user_inputs_dict

In [ ]:
from parsers.PrimitiveParsers import YamlFileParser
from scripts.script_generate_with_new_architecture import generate_with_new_architecture
import shutil

# Toggle depending on whether you exported from PhLynx
use_phlynx_export = True
phlynx_cellml_path = downloads_dir / f"{file_prefix}.cellml"  # TODO: update filename if needed

# Parse and expand user inputs (sets generated_models_subdir, model_path, etc.)
parser = YamlFileParser()
inp_data_dict = parser.parse_user_inputs_file(
    user_inputs_dict,
    obs_path_needed=False,
    do_generation_with_fit_parameters=False,
)

if use_phlynx_export:
    generated_models_subdir = Path(inp_data_dict["generated_models_subdir"])
    generated_models_subdir.mkdir(parents=True, exist_ok=True)
    target_cellml_path = generated_models_subdir / f"{file_prefix}.cellml"
    shutil.copy2(phlynx_cellml_path, target_cellml_path)
    inp_data_dict["model_path"] = str(target_cellml_path)
else:
    # Generate directly from resources CSVs
    success = generate_with_new_architecture(False, inp_data_dict)
    if not success:
        raise RuntimeError("Model generation failed")
    # Re-parse to refresh derived paths
    inp_data_dict = parser.parse_user_inputs_file(
        inp_data_dict,
        obs_path_needed=False,
        do_generation_with_fit_parameters=False,
    )

model_path = Path(inp_data_dict["model_path"])
model_path

## 3) Load the model with SimulationHelper

Use the solver wrapper to load the CellML model and prepare a simulation helper.

In [ ]:
from solver_wrappers import get_simulation_helper

# Simulation settings
sim_time = 2.0
pre_time = 20.0
solver = inp_data_dict.get("solver")
solver_info = inp_data_dict.get("solver_info", {})

SimHelper = get_simulation_helper(
    solver=solver,
    model_type=inp_data_dict.get("model_type"),
    model_path=str(model_path),
)

sim_helper = SimHelper(
    str(model_path),
    dt=inp_data_dict["dt"],
    sim_time=sim_time,
    solver_info=solver_info,
    pre_time=pre_time,
)

# Run once and plot a few representative variables
sim_helper.run()
variables_to_plot = [
    "venous_svc/u",
    "aortic_root/v",
    "aortic_root/u",
    "heart/u_rv",
]

y = sim_helper.get_results(variables_to_plot, flatten=True)
t = sim_helper.tSim - pre_time

plot_dir = Path(param_id_output_dir) / "quicklooks"
plot_dir.mkdir(parents=True, exist_ok=True)

fig, axs = plt.subplots(2, 2, sharex=True, figsize=(8, 6))
axs = axs.flatten()
for idx, (ax, series, name) in enumerate(zip(axs, y, variables_to_plot)):
    ax.plot(t, series)
    ax.set_title(name)
    ax.set_xlabel("Time [s]")

plt.tight_layout()
plt.savefig(plot_dir / "uncalibrated_outputs.png")
plt.show()

## 4) Plot ground-truth data vs uncalibrated outputs

Load the ground-truth data and compare it to the uncalibrated simulation results.

In [ ]:
# TODO: update with your ground-truth data file
# Example assumes a CSV with a time column and measurement columns

ground_truth_csv = resources_dir / f"{file_prefix}_ground_truth.csv"
if ground_truth_csv.exists():
    gt = pd.read_csv(ground_truth_csv)
    time_col = "time"  # TODO: set correct column name
    gt_var = "aortic_root/u"  # TODO: set correct variable column

    plt.figure(figsize=(6, 4))
    plt.plot(gt[time_col], gt[gt_var], label="ground truth")
    plt.plot(t, y[2], label="uncalibrated")
    plt.xlabel("Time [s]")
    plt.ylabel(gt_var)
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print(f"Ground-truth file not found: {ground_truth_csv}")

## 5) Add arm vessels in PhLynx (manual)

- Add arm vessels in PhLynx.
- Regenerate the CellML model.
- If you exported a new CellML file, re-run the copy/generation cell above.

In [ ]:
from utilities.obs_data_helpers import ObsDataCreator

# TODO: update with your real data source
# Example: data = pd.read_csv(resources_dir / "my_observations.csv")
# time = data["time"].values

obs_data_creator = ObsDataCreator()

# Protocol info (edit to match your experiment design)
pre_times = [pre_time]
sim_times = [[sim_time]]
params_to_change = {}
obs_data_creator.add_protocol_info(pre_times, sim_times, params_to_change)

# Prediction items (edit to match outputs you compare)
obs_data_creator.add_prediction_item("aortic_root/u", "Pa", 0)

# Data items (fill with your actual observation series)
# Example placeholder (replace with real data)
entry = {
    "variable": "aortic_root/u",
    "name_for_plotting": "P_ao",
    "data_type": "series",
    "operation": None,
    "operands": ["aortic_root/u"],
    "unit": "Pa",
    "weight": 1.0,
    "value": [],  # TODO: fill with data values
    "std": [],    # TODO: fill with std values
    "obs_dt": inp_data_dict["dt"],
    "experiment_idx": 0,
    "subexperiment_idx": 0,
}
obs_data_creator.add_data_item(entry)

obs_data_path = resources_dir / f"{file_prefix}_obs_data.json"
obs_data_creator.dump_to_path(str(obs_data_path))
obs_data_path

## 6) Create params_for_id.csv from Python

Define the parameter bounds for identification in a CSV file. Use the same schema as `resources/3compartment_params_for_id.csv`.


In [ ]:
# TODO: replace with your full parameter set
params_for_id = [
    {
        "vessel_name": "global",
        "param_name": "q_lv_init",
        "param_type": "const",
        "min": 200e-6,
        "max": 1500e-6,
        "name_for_plotting": "q_{sbv}",
    },
    {
        "vessel_name": "aortic_root",
        "param_name": "C",
        "param_type": "const",
        "min": 1e-9,
        "max": 5e-8,
        "name_for_plotting": "C_{ao}",
    },
]

params_df = pd.DataFrame(params_for_id)
params_for_id_path = resources_dir / f"{file_prefix}_params_for_id.csv"
params_df.to_csv(params_for_id_path, index=False)

# Track for downstream steps
inp_data_dict["params_for_id_path"] = str(params_for_id_path)
params_for_id_path


## 7) Sensitivity analysis

Run Sobol sensitivity analysis over a large parameter set and save outputs to a defined directory.


In [ ]:
from sensitivity_analysis.sensitivityAnalysis import SensitivityAnalysis

# Ensure paths are wired into inputs
inp_data_dict["param_id_obs_path"] = str(obs_data_path)

sa_output_dir = Path(param_id_output_dir) / "sensitivity" / file_prefix
sa_output_dir.mkdir(parents=True, exist_ok=True)

sa_options = {
    "method": "sobol",
    "sample_type": "saltelli",
    "num_samples": 256,
    "output_dir": str(sa_output_dir),
}

model_out_names = ["aortic_root/u", "aortic_root/v"]  # TODO: update

sa_agent = SensitivityAnalysis(
    model_path=str(model_path),
    model_type=inp_data_dict["model_type"],
    file_name_prefix=file_prefix,
    DEBUG=inp_data_dict["DEBUG"],
    param_id_output_dir=str(param_id_output_dir),
    resources_dir=str(resources_dir),
    model_out_names=model_out_names,
    solver_info=solver_info,
    dt=inp_data_dict["dt"],
    ga_options=inp_data_dict.get("optimiser_options", {}),
    param_id_obs_path=str(obs_data_path),
    params_for_id_path=str(params_for_id_path),
)

sa_agent.run_sensitivity_analysis(sa_options)
sa_output_dir


## 8) Choose influential parameters

Review the sensitivity analysis outputs (plots/arrays) to choose the most influential parameters (e.g., top 2).


In [ ]:
# TODO: inspect SA outputs in sa_output_dir and pick top parameters
# Example placeholder list:
most_influential_params = [
    "global/q_lv_init",
    "aortic_root/C",
]
most_influential_params


## 9) Parameter identification (genetic algorithm)

Use the genetic algorithm to calibrate model parameters.

$$
\min_{\theta \in \Theta} \; \sum_{i=1}^{N} w_i \lVert y_i^{model}(\theta) - y_i^{obs} \rVert^2
$$

> Replace the objective above with your formal calibration problem as needed.


In [ ]:
from param_id.paramID import CVS0DParamID

# Optimiser options (adjust as needed)
optimiser_options = {
    "num_calls_to_function": 10000,
    "cost_convergence": 0.001,
    "max_patience": 10,
    "cost_type": "MSE",
}

param_id = CVS0DParamID(
    model_path=str(model_path),
    model_type=inp_data_dict["model_type"],
    param_id_method=inp_data_dict["param_id_method"],
    mcmc_instead=False,
    file_prefix=file_prefix,
    params_for_id_path=str(params_for_id_path),
    param_id_obs_path=str(obs_data_path),
    sim_time=sim_time,
    pre_time=pre_time,
    solver_info=solver_info,
    dt=inp_data_dict["dt"],
    optimiser_options=optimiser_options,
    DEBUG=inp_data_dict["DEBUG"],
    param_id_output_dir=str(param_id_output_dir),
    resources_dir=str(resources_dir),
)

# Genetic algorithm settings (if applicable)
if inp_data_dict["param_id_method"] == "genetic_algorithm":
    param_id.set_genetic_algorithm_parameters(optimiser_options["num_calls_to_function"])

# Run calibration
param_id.run()

# Plot best fit
param_id.simulate_with_best_param_vals()
param_id.plot_outputs()


## 10) Review calibration outputs

The calibration process saves plots and arrays in the parameter ID output directory. Use this section to generate additional figures.


In [ ]:
output_dir = Path(param_id.output_dir)
print("Outputs saved to:", output_dir)

# Example: load best parameters
best_params_path = output_dir / "best_param_vals.npy"
if best_params_path.exists():
    best_params = np.load(best_params_path)
    best_params
else:
    print("best_param_vals.npy not found")

# TODO: add custom plots using saved outputs in output_dir


## 11) Calculate PWV with the 0D model

Add your preferred PWV computation here.


In [ ]:
# TODO: compute PWV from 0D outputs
# pwv_0d = compute_pwv(sim_helper, ...)
# pwv_0d


## 12) Switch selected vessels to 1D and generate C++ model

Update vessel arrays for 1D coupling and regenerate a C++ model.


In [ ]:
# TODO: update vessel array to set specific vessels to 1D in PhLynx

inp_data_dict_cpp = dict(inp_data_dict)
inp_data_dict_cpp.update({
    "model_type": "cpp",
    "couple_to_1d": True,
    "cpp_generated_models_dir": str(generated_models_dir / file_prefix),
    "cpp_1d_model_config_path": "/path/to/input1d.dat",  # TODO: update
})

# Generate C++ model with 1D coupling
success_cpp = generate_with_new_architecture(False, inp_data_dict_cpp)
if not success_cpp:
    raise RuntimeError("C++ model generation failed")


## 13) Run the C++ model

Use `subprocess` to run the compiled binary for the 0D-1D hybrid.


In [ ]:
import subprocess

# TODO: update binary path and arguments
cpp_run_dir = Path(inp_data_dict_cpp.get("cpp_generated_models_dir", generated_models_dir / file_prefix))
cpp_binary = cpp_run_dir / "run_model"  # placeholder

if cpp_binary.exists():
    result = subprocess.run([str(cpp_binary)], cwd=str(cpp_run_dir), capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)
else:
    print(f"C++ binary not found: {cpp_binary}")


## 14) Load and plot 0D-1D hybrid results

Load the hybrid outputs and compare them to the 0D results.


In [ ]:
# TODO: update with real hybrid output file(s)
hybrid_output_csv = cpp_run_dir / "hybrid_outputs.csv"

if hybrid_output_csv.exists():
    hybrid = pd.read_csv(hybrid_output_csv)
    time_col = "time"  # TODO: update
    hybrid_var = "aortic_root/u"  # TODO: update

    plt.figure(figsize=(6, 4))
    plt.plot(t, y[2], label="0D")
    plt.plot(hybrid[time_col], hybrid[hybrid_var], label="0D-1D hybrid")
    plt.xlabel("Time [s]")
    plt.ylabel(hybrid_var)
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print(f"Hybrid output not found: {hybrid_output_csv}")


## 15) PWV comparison (0D vs 0D-1D)

Compute PWV for the hybrid outputs and compare with the 0D model.


In [ ]:
# TODO: compute and compare PWV for hybrid outputs
# pwv_hybrid = compute_pwv_from_hybrid(hybrid)
# print("PWV 0D:", pwv_0d)
# print("PWV hybrid:", pwv_hybrid)


## 16) Concise-call hints (commented)

Below are shorter, user-friendly call patterns you can aim to implement as wrappers.


In [ ]:
# generate_model(file_prefix, model_type="cellml_only")
# sim = run_simulation(model_path, dt=0.01, sim_time=2.0, pre_time=20.0)
# plot_outputs(sim, ["aortic_root/u", "aortic_root/v"])
# obs_path = create_obs_data(data, protocol_info, predictions)
# params_path = create_params_for_id(params_df)
# sa_results = run_sensitivity(model_path, obs_path, params_path, output_dir)
# best_params = pick_top_parameters(sa_results, top_k=2)
# param_id = calibrate_model(model_path, obs_path, params_path, method="genetic_algorithm")
# run_cpp_model(cpp_run_dir)
# hybrid = load_hybrid_outputs(cpp_run_dir)
# compare_outputs(uncalibrated, hybrid)
# compare_pwv(pwv_0d, pwv_hybrid)


## 6) Save artifacts

Persist calibrated parameters, summary metrics, and figures.


In [ ]:
# TODO: save outputs
# (output_dir / "calibration_results.json").write_text(result.to_json())
